In [1]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
optns = []
for ls in list(set(spacex_df['Launch Site'])) + ['ALL']:
    optns.append({'label': ls, 'value': ls})

app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown', options=optns, value='ALL', placeholder='Select a Launch Site here', searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider', min=0, max=10000, step=1000, value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def update_site_pie_chart(site):
    if site == 'ALL':
        return px.pie(spacex_df, values='class', names='Launch Site', title='Total Success Launches By Site')

    return px.pie(spacex_df[spacex_df['Launch Site'] == site].groupby(['Launch Site', 'class']).size().reset_index(name='class count'), values='class count', names='class', title='Total Success Launches For ' + site)

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'), 
    Input(component_id="payload-slider", component_property="value")]
)
def update_success_payload_scatter_chart(site, payload):
    mask = (spacex_df['Payload Mass (kg)'] > payload[0]) & (spacex_df['Payload Mass (kg)'] < payload[1])

    if site == 'ALL':
        return px.scatter(spacex_df[mask], x='Payload Mass (kg)', y='class', color='Booster Version Category', title='Payload Success Rate For All')

    return px.scatter(spacex_df[mask & (spacex_df['Launch Site'] == site)], x='Payload Mass (kg)', y='class', color='Booster Version Category', title='Payload Success Rate For ' + site)

# Run the app
if __name__ == '__main__':
    app.run_server()

ModuleNotFoundError: No module named 'dash'